In [3]:
from datasets import load_dataset

# This can take a few minutes to load, so grab a coffee or tea while you wait!
raw_datasets = load_dataset("code_search_net.py", "python")

RuntimeError: Dataset scripts are no longer supported, but found code_search_net.py